In [23]:
import scrapy
import requests
from scrapy.http import TextResponse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from scrapy.selector import Selector
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from bs4 import BeautifulSoup
import urllib.request
import warnings
warnings.filterwarnings("ignore")

In [24]:
# from fake_useragent import UserAgent
# url = "https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from=2022&release_year_until=2022"
# headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
# req = requests.get(url, headers=headers)
# response = TextResponse(req.url, body=req.text, encoding="utf-8") 
# response

In [25]:
# #해당연도의 영화 총 개수
# import re

# total_movie_cnt = response.xpath('//*[@id="base"]/div[3]/div/div[1]/div/div[1]/div[1]/text()').get()
# numbers = int(re.sub(r'[^0-9]','', total_movie_cnt))
# print(numbers)

In [26]:
def get_chrome_driver():
    # 1. 크롬 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    WINDOW_SIZE = "1920,1080"
    
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument(f"--window-size={WINDOW_SIZE}")
    # 2. driver 생성하기
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), # 가장 많이 바뀐 부분
        options=chrome_options
    )
    
    return driver

In [27]:
driver = get_chrome_driver()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Kims\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [28]:
urls = []
for year in map(str,range(2005,2006)):
    url = 'https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from={}&release_year_until={}'.format(year, year)
    urls.append(url)

In [29]:
for url in urls:
    driver.get(url)
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 1초 대기
        time.sleep(1)
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height        

    # detail_links = response.css('div.title-list-grid__item > a[href^=https]')
    # detail_links = list(map(response.urljoin,detail_links))
    # # len(detail_links)
    # detail_links
        elem = driver.find_elements_by_css_selector('div.title-list-grid div.title-list-grid__item a.title-list-grid__item--link')
        linkUrls = []
        for e in elem:
            linkUrl = e.get_attribute('href')
            linkUrls.append(linkUrl)    

In [30]:
len(linkUrls)

44

In [47]:
title_list = []
openingdate_list = []
just_rating_list = []
imdb_rating_list = []
genre_list = []
runtime_list = []
director_list = []
synopsis_list = []
actors_list = []
poster_list = []

In [48]:
for i in range(0,len(linkUrls)):
    url = linkUrls[i]
    headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
    req = requests.get(url, headers=headers)
    response = TextResponse(req.url, body=req.text, encoding="utf-8")
    response

    
    title_kor = response.css("div.title-block > div > h1::text").get().strip()        
    try:
        opening_date = response.css("div.title-block > div > span::text").get().replace("(","").replace(")","").strip()
    except:
        opening_date = None
    try:
        just_rating = response.css('div.detail-infos > div > div > div > a::text')[0].get().strip()
    except:
        just_rating = None
    try:
    
        imdb_rating = response.css('div.detail-infos > div > div > div > a::text')[1].get().strip().split(" ")[0]
    except:
        imdb_rating = None   
    genre = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div/div[2]/span/text()').getall()     # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
    genre = ",".join(genre).replace(" ","")
    genre = genre
    runtime = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[3]/div[2]/text()').getall()
    runtime = ",".join(runtime).replace(" ","")
    runtime = runtime
    director = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/div[4]/div[2]/span/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음..
    director = ",".join(director).replace(" ","")
    director = director
    actors = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[3]/div/div/a/text()').getall() # 뽑아낸 값이 리스트형태로 뽑힘 str 형태로 바꿔야함 모르겠음...
    actors = ",".join(actors).replace(" ","")
    actors = actors
    
    try:
        synopsis = response.xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]/div[4]/p/span/text()').get()
    except:
        synopsis= None

    poster = response.xpath('//*[@id="base"]/div[2]/div/div[1]/div/aside/div/div/picture').css('img').attrib['data-src']
 
    
    


    title_list.append(title_kor)
    openingdate_list.append(opening_date)
    just_rating_list.append(just_rating)
    imdb_rating_list.append(imdb_rating)
    genre_list.append(genre)
    runtime_list.append(runtime)
    director_list.append(director)
    synopsis_list.append(synopsis)
    actors_list.append(actors)
    poster_list.append(poster)

In [49]:
len(title_list)

44

In [50]:
len(poster_list)

44

In [51]:
imdb_rating_list

['5.5',
 '7.0',
 '7.8',
 '6.7',
 '7.7',
 '6.9',
 '6.5',
 '6.4',
 '7.4',
 '7.2',
 '7.0',
 '6.5',
 '6.5',
 '7.6',
 '7.4',
 '6.9',
 '6.2',
 '7.7',
 '7.3',
 '5.9',
 '6.6',
 '6.7',
 '7.7',
 '6.8',
 '7.2',
 '5.9',
 '6.4',
 '7.0',
 None,
 '7.1',
 '6.0',
 '6.3',
 '7.4',
 '6.8',
 '5.8',
 '7.0',
 '6.6',
 '6.0',
 '6.3',
 '6.9',
 '5.8',
 '6.6',
 '6.2',
 '6.0']

In [53]:
print("제목:",title_kor, "개봉일:",opening_date,"저스트와치 평점:", just_rating,"IMDB 평점:",imdb_rating, "장르:",genre, "재생시간:",runtime, "감독:",director,"출연자:", actors, "시놉시스:",synopsis)

제목: 공공의 적 2 개봉일: 2005 저스트와치 평점: 72% IMDB 평점: 6.0 장르: 스릴러,드라마,액션 재생시간: 2시간28분 감독: KangWoo-suk 출연자: SolKyung-gu,JungJoon-ho,KangShin-il,ParkGeun-hyung,ByunHee-bong,ParkSang-wook,UhmTae-woong,JeongGyu-su,ChoiJung-woo,ParkWoong 시놉시스: 검찰청 최고의 꼴통검사 강철중, 룸싸롱은 주로 쳐들어 가기 위해 자주 들르고 책상머리의 서류철 들여다보기 보다 컵라면 우물거리며 잠복근무가 체질인 그는 나쁜 짓 하는 놈 잡기 위해 부하와 동료의 피를 볼 순 없다는 일념으로 총기류 사용도 마다하지 않는 다혈질 검사다. 그렇지만 어제도 오늘도 그의 하루는 늘 나름대로 스마일~로 시작된다.  그런 그에게 명선 재단 이사장 한상우 사건이 접수되고 특유의 기질로 나쁜 냄새를 직감한 강검사, 자기 담당도 아닌 사건을 조사하기 시작한다. 명선고교는 다름아닌 강검사의 모교이며 학교 재단 이사장의 둘째 아들인 한상우 또한 자신과 3년 동안 같은 반에서 수학한 동기동창이었던 것. 재단을 물려받기로 되어 있던 명선 재단 큰 아들의 사고로 인해 재단 이사장으로 급부상한 한상우, 때마침 명선 재단 큰 아들이 숨을 거두고 그 죽음에 대한 의문이 제기되면서 강철중 검사의 수사는 큰 파문을 불러 일으킨다. 이에 철중은 조사인의 자격으로 상우를 검찰청에서 마주하게 되는데.


In [55]:
import pandas as pd
# df = pd.DataFrame({
#     "제목":title_list,"개봉일":openingdate_list,"저스트와치 평점":just_rating_list,"장르":genre_list,"재생시간":runtime_list, "감독":director_list,"시놉시스":synopsis_list})
# df.tail(5)
df = pd.DataFrame({
    "제목":title_list,"개봉일":openingdate_list, "imdb평점":imdb_rating_list})
df

,제목,개봉일,imdb평점
0,Kureyon Shinchan: Densetsu o Yobu Buriburi: Sa...,2005,5.5
1,콘스탄틴,2005,7.0
2,오만과 편견,2005,7.8
3,찰리와 초콜릿 공장,2005,6.7
4,브로크백 마운틴,2005,7.7
5,마다가스카,2005,6.9
6,우주전쟁,2005,6.5
7,연애의 목적,2005,6.4
8,유령 신부,2005,7.4
9,디센트,2005,7.2


In [12]:
driver.quit()


In [65]:
# csv파일 읽어오기
import pandas as pd
# csv_test = pd.read_csv("C:/Users/Kims/Desktop/HK_Project_Team5/movies.csv")
csv_test = pd.read_csv(r'C:\Workspace\AtHome\Justwath_Scrapy\moviedata\MovieCrawlerFile_disney.csv')


In [113]:
csv_test.tail(5)

,actors,director,genre,imdb_rating,just_rating,opening_date,posterLink,runtime,synopsis,title_kor
973,"BenAffleck,RosamundPike,NeilPatrickHarris,Tyle...",DavidFincher,"스릴러,드라마",8.1,88%,2014.0,https://images.justwatch.com/poster/177088991/...,2시간29분,모두가 부러워하는 삶을 살아가는 완벽한 커플인 닉과 에이미. 결혼 5주년 기념일 아...,나를 찾아줘
974,"TaronEgerton,ColinFirth,MarkStrong,SamuelL.Jac...",MatthewVaughn,"코미디,스릴러,액션",7.7,85%,2014.0,https://images.justwatch.com/poster/187271737/...,2시간9분,"높은 IQ, 주니어 체조대회 2년 연속 우승! 그러나 학교 중퇴, 해병대 중도 하차...",킹스맨: 시크릿 에이전트
975,"MattDamon,JessicaChastain,KristenWiig,JeffDani...",RidleyScott,"드라마,SF",8.0,87%,2015.0,https://images.justwatch.com/poster/176289750/...,2시간22분,NASA 소속의 아레스 3 탐사대는 화성에 도착한 지 6일째 되는 날 거대한 모래 ...,마션
976,"MichaelFassbender,KatherineWaterston,BillyCrud...",RidleyScott,"공포,SF,스릴러",6.4,71%,2017.0,https://images.justwatch.com/poster/8573813/s592,2시간3분,2천명의 승객과 12명의 승무원을 실은 커버넌트호는 새로운 행성의 개척과 이주를 위...,에이리언: 커버넌트
977,"KeriRussell,JessePlemons,JeremyThomas,GrahamGr...",ScottCooper,"공포,판타지,스릴러,드라마,액션,범죄",5.9,71%,2021.0,https://images.justwatch.com/poster/257452810/...,1시간39분,"오리건의 작은 마을 선생 (케리 러셀), 지역 보안관인 그녀의 오빠 (제시 플레몬스...",앤틀러스


In [158]:
csv_test["posterLink"][900].split("poster/")[1].split("/")[0]

'185385933'

In [194]:
link = csv_test["posterLink"]
len(link)

978

In [197]:
len(link)

978

In [195]:
xs = []
for i in range(0,len(link)):
    x = link[i]
    xs.append(x)
len(xs)

978

In [198]:
for i in range(0,len(link)):
    x = csv_test["posterLink"][i]
    number = re.findall('\d+', x)[0]
    


TypeError: expected string or bytes-like object

In [186]:
import re
x = csv_test["posterLink"][0]
number = re.findall('\d+', x)[0]

number

'18697230'

In [ ]:
urls = []
for year in map(str,range(1962,2023)):
    ss = 'https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix?content_type=movie&release_year_from={}&release_year_until={}'.format(year, year)
    urls.append(ss)

urls

In [20]:
x = "7.3 (27k)"

In [22]:
x.split(" ")[0]

'7.3'

SyntaxError: invalid syntax (Temp/ipykernel_20856/234169970.py, line 1)